In [1]:
import logging

import probtorch
import torch

import combinators
import filtering
import hmm
import smc
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi, _ = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
inference = smc.ParticleTrace(100)

In [12]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [13]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(inference, generative)

In [14]:
z0, mu, sigma, pi, pi0 = init_hmm(T=50)

In [15]:
hmm_step = hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [16]:
hmm_run.condition(inference, generative)

In [17]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [18]:
hmm_step.backward_pass(T=50)